In [11]:
# All imports will be here:
import pandas as pd
import numpy as np
from utils import import_and_transform
from utils import evaluate_model
from utils import aggregate
from utils import evaluate_model_log_reg
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [12]:

def import_and_transform(data):
    """Import and basic preprocessing only."""
    if isinstance(data, str):
        df = pd.read_parquet(data)
    else:
        df = data
    
    df = df[df['userId'] != '']
    df['userId'] = df['userId'].astype(int)
    df["gender"] = df["gender"].map({'F': 0, 'M': 1})
    df["level"] = df["level"].map({'free': 0, 'paid': 1})
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    df['registration'] = pd.to_datetime(df['registration'])
    
    df['session_length'] = df.groupby(['userId', 'sessionId'])['ts'].transform(
        lambda x: (x.max() - x.min()).total_seconds()
    )
    df['song_played'] = (df['page'] == 'NextSong').astype(int)
    
    return df


def aggregate_features(data, observation_end):
    """Calculate features using only data up to observation_end."""
    observation_end = pd.Timestamp(observation_end)
    
    user_df = data.groupby('userId').agg({
        'gender': 'first',
        'registration': 'first',
        'level': lambda x: x.mode().iloc[0] if not x.mode().empty else 0,
        'sessionId': 'nunique',
        'itemInSession': 'max',
        'ts': ['min', 'max'],
        'session_length': 'mean',
        'song_played': 'sum',
        'artist': pd.Series.nunique,
        'length': 'sum'
    }).reset_index()
    
    user_df.columns = ['userId', 'gender', 'registration', 'level',
                       'num_sessions', 'max_item_in_session', 'ts_min', 'ts_max', 
                       'avg_session_length_seconds', 'num_songs_played', 
                       'unique_artists', 'total_length']
    
    user_df['days_active'] = (observation_end - user_df['ts_min']).dt.days
    user_df['membership_length'] = (observation_end - user_df['registration']).dt.days
    user_df['days_since_last_activity'] = (observation_end - user_df['ts_max']).dt.days
    user_df['songs_per_day'] = user_df['num_songs_played'] / (user_df['days_active'] + 1)
    user_df['sessions_per_day'] = user_df['num_sessions'] / (user_df['days_active'] + 1)
    
    user_df = user_df.fillna(0)
    user_df.set_index('userId', inplace=True)
    
    return user_df


def get_churned_users(df, start_date, end_date):
    """Get users who churned between start_date and end_date."""
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)
    
    cancellations = df[df['page'] == 'Cancellation Confirmation']
    churned = cancellations[(cancellations['ts'] > start) & 
                           (cancellations['ts'] <= end)]['userId'].unique()
    return set(churned)


# Load training data
df_train = import_and_transform('Data/train.parquet')

# Prepare test data
df_test = import_and_transform('Data/test.parquet')



In [13]:
from utils import aggregate_features_improved
# Createing observation dates every 5 days
# Create multiple training samples with sliding window
training_dates = pd.date_range("2018-10-15", "2018-11-05", freq="5D")

X_train_list = []
y_train_list = []

# For each observ date, we create a separate training sample:
for obs_date in training_dates:

    # Filtering data up to the observation date
    df_obs = df_train[df_train["ts"] <= obs_date]
    features = aggregate_features_improved(df_obs, obs_date)

    # Creating a 10 day window after the obervation date
    # And we identify who churned in that period
    window_end = obs_date + pd.Timedelta(days=10)
    churned_users = get_churned_users(df_train, obs_date, window_end)

    # 1 if they churned in the next 10 days, 0 otherwise
    labels = pd.Series(
        features.index.isin(churned_users).astype(int),
        index=features.index,
        name="churned",
    )

    X_train_list.append(features)
    y_train_list.append(labels)

    print(
        f"Date of the observation: {obs_date.date()}, with {len(features)} users, and a {labels.mean():.2%} churn rate"
    )

# We combine all observation windows:
X_train_combined = pd.concat(X_train_list)
y_train_combined = pd.concat(y_train_list)

# Drop non-numeric columns
feature_cols = X_train_combined.select_dtypes(include=[np.number]).columns
feature_cols = [
    c
    for c in feature_cols
    if c not in ["registration", "ts_min", "ts_max", "total_length"]
]

X_train_final = X_train_combined[feature_cols]

test_features = aggregate_features_improved(df_test, "2018-11-20")
X_test = test_features[feature_cols]

ImportError: cannot import name 'aggregate_features_improved' from 'utils' (c:\Users\andre\Documents\university\MASTER\Polytechnique\Python_for_data_science\DataScienceProject\utils.py)

In [ ]:
!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, f1_score

# Calculate scale_pos_weight
scale_pos_weight = (y_train_combined == 0).sum() / (y_train_combined == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")

# MINIMAL GRID - Focus on most important parameters
param_grid_xgb = {
    'max_depth': [3, 5, 7],                    # Tree depth (most important!)
    'learning_rate': [0.05, 0.1],              # Step size (most important!)
    'n_estimators': [200, 300],                # Number of trees
    'scale_pos_weight': [scale_pos_weight]     # Fixed based on class imbalance
}

# Create XGBoost with good defaults for other parameters
xgb_base = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    subsample=0.8,              # Use 80% of data per tree (reduces overfitting)
    colsample_bytree=0.8,       # Use 80% of features per tree
    tree_method='hist',         # Much faster for large datasets
    n_jobs=-1
)

# Reduce CV folds for speed
grid_search_xgb = GridSearchCV(
    xgb_base,
    param_grid_xgb,
    cv=3,                       # 3 folds instead of 5 (faster!)
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

print("\n🔍 Starting XGBoost Grid Search (this may take 5-10 minutes)...")
grid_search_xgb.fit(X_train_final, y_train_combined)

print(f"\n✅ Grid Search Complete!")
print(f"Best params: {grid_search_xgb.best_params_}")
print(f"Best CV ROC-AUC score: {grid_search_xgb.best_score_:.4f}")

# Get the best model
best_xgb = grid_search_xgb.best_estimator_

evaluate_model(best_xgb, X_test, 0.5, file_out='xgboost.csv')

Scale pos weight: 21.53

🔍 Starting XGBoost Grid Search (this may take 5-10 minutes)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits

✅ Grid Search Complete!
Best params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'scale_pos_weight': 21.53161652218256}
Best CV ROC-AUC score: 0.7044
Base predicted churn: 51.07%
Predicted churn at 0.5 threshold: 51.07%
Submission saved to xgboost.csv


In [9]:
# More comprehensive grid based on your best params
param_grid_xgb_advanced = {
    'max_depth': [2, 3, 4],                    # Your best was 3, try neighbors
    'learning_rate': [0.03, 0.05, 0.07],       # Your best was 0.05, fine-tune around it
    'n_estimators': [200, 300, 400],           # Your best was 200, try more trees
    'min_child_weight': [1, 3, 5],             # Prevent overfitting
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction
    'subsample': [0.7, 0.8, 0.9],              # Row sampling
    'colsample_bytree': [0.7, 0.8, 0.9],       # Column sampling
    'reg_alpha': [0, 0.01, 0.1],               # L1 regularization
    'reg_lambda': [1, 1.5, 2],                 # L2 regularization
    'scale_pos_weight': [scale_pos_weight]
}

xgb_advanced = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    tree_method='hist',
    n_jobs=-1
)

# Use RandomizedSearchCV instead of GridSearchCV for speed
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    xgb_advanced,
    param_grid_xgb_advanced,
    n_iter=50,              # Test 50 random combinations
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X_train_final, y_train_combined)
print(f"Best params: {random_search.best_params_}")
print(f"Best CV ROC-AUC: {random_search.best_score_:.4f}")

best_model_2 = random_search.best_estimator_
evaluate_model(best_model_2, X_test, 0.5, file_out ='xgboost_2.csv')

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\andre\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'subsample': 0.7, 'scale_pos_weight': 21.53161652218256, 'reg_lambda': 1, 'reg_alpha': 0.01, 'n_estimators': 200, 'min_child_weight': 3, 'max_depth': 2, 'learning_rate': 0.03, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best CV ROC-AUC: 0.6904
Base predicted churn: 58.26%
Predicted churn at 0.5 threshold: 58.26%
Submission saved to xgboost_2.csv
